# 1. Import packages

In [1]:
import sys
sys.path.append('..')
from modules import tuning, preprocess, tracking, benchmarking

Invoking __init__.py for modules


# 2. Create modified dataset

In [2]:
!python ../scripts/create-dataset.py

Invoking __init__.py for modules


# 3. Load dataset

In [3]:
df = preprocess.load_csv_data('../dataset/New_Occupancy_Estimation.csv')
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


# 4. Tuning Hyperparameters: Cross Validation

In [4]:
# Split dataset into features and output
X = preprocess.get_features(df)
y = preprocess.get_output(df)

# Split dataset into training and test
X_train, X_test, y_train, y_test = preprocess.split_dataset(X, y)

## 1. Logistic Regression (LR)

In [5]:
LR_best_estimator = tuning.tuning_with_grid('LR', X_train, y_train, X_test, y_test, 5)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV] END estimator__C=0.001, estimator__penalty=l1, estimator__solver=lbfgs; total time=   0.0s
[CV] END estimator__C=0.001, estimator__penalty=l1, estimator__solver=lbfgs; total time=   0.0s
[CV] END estimator__C=0.001, estimator__penalty=l1, estimator__solver=lbfgs; total time=   0.0s
[CV] END estimator__C=0.001, estimator__penalty=l1, estimator__solver=lbfgs; total time=   0.0s
[CV] END estimator__C=0.001, estimator__penalty=l1, estimator__solver=lbfgs; total time=   0.0s
[CV] END estimator__C=0.001, estimator__penalty=l1, estimator__solver=liblinear; total time=   0.0s
[CV] END estimator__C=0.001, estimator__penalty=l1, estimator__solver=liblinear; total time=   0.0s
[CV] END estimator__C=0.001, estimator__penalty=l1, estimator__solver=liblinear; total time=   0.0s
[CV] END estimator__C=0.001, estimator__penalty=l1, estimator__solver=liblinear; total time=   0.0s
[CV] END estimator__C=0.001, estimator__penalty=l1, estima

/home/misanchz/miniconda3/envs/TFG/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/home/misanchz/miniconda3/envs/TFG/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/misanchz/miniconda3/envs/TFG/lib/python3.10/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/misanchz/miniconda3/envs/TFG/lib/python3.10/site-packages/sklearn/linear_model/_l

## 2. Random Forest (RF)

In [6]:
RF_best_estimator = tuning.tuning_with_randomized('RF', X_train, y_train, X_test, y_test, 5)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END bootstrap=False, max_depth=3, max_features=log2, max_leaf_nodes=3, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.4s
[CV] END bootstrap=False, max_depth=3, max_features=log2, max_leaf_nodes=3, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.3s
[CV] END bootstrap=False, max_depth=3, max_features=log2, max_leaf_nodes=3, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.3s
[CV] END bootstrap=False, max_depth=3, max_features=log2, max_leaf_nodes=3, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.6s
[CV] END bootstrap=False, max_depth=3, max_features=log2, max_leaf_nodes=3, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.4s
[CV] END bootstrap=False, max_depth=6, max_features=sqrt, max_leaf_nodes=6, min_samples_leaf=2, min_samples_split=5, n_estimators=25; total time=   0.1s
[CV] END bo

## 3. Support Vector Machine (SVM)

In [7]:
SVM_best_estimator = tuning.tuning_with_grid('SVM', X_train, y_train, X_test, y_test, 5)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf; total time=   0.7s
[CV] END estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf; total time=   0.7s
[CV] END estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf; total time=   0.7s
[CV] END estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf; total time=   0.7s
[CV] END estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf; total time=   0.7s
[CV] END estimator__C=0.1, estimator__gamma=0.1, estimator__kernel=rbf; total time=   0.2s
[CV] END estimator__C=0.1, estimator__gamma=0.1, estimator__kernel=rbf; total time=   0.3s
[CV] END estimator__C=0.1, estimator__gamma=0.1, estimator__kernel=rbf; total time=   0.2s
[CV] END estimator__C=0.1, estimator__gamma=0.1, estimator__kernel=rbf; total time=   0.3s
[CV] END estimator__C=0.1, estimator__gamma=0.1, estimator__kernel=rbf; total time=   0.2s
[CV] END estimator__C=0.1, estimator__

## 4. Multilayer Perceptron (MLP)

In [8]:
MLP_best_estimator = tuning.tuning_with_randomized('MLP', X_train, y_train, X_test, y_test, 5)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END estimator__activation=tanh, estimator__alpha=0.05, estimator__hidden_layer_sizes=(50, 100, 50), estimator__learning_rate=invscaling, estimator__solver=lbfgs; total time=   2.1s
[CV] END estimator__activation=tanh, estimator__alpha=0.05, estimator__hidden_layer_sizes=(50, 100, 50), estimator__learning_rate=invscaling, estimator__solver=lbfgs; total time=   1.2s
[CV] END estimator__activation=tanh, estimator__alpha=0.05, estimator__hidden_layer_sizes=(50, 100, 50), estimator__learning_rate=invscaling, estimator__solver=lbfgs; total time=   1.3s
[CV] END estimator__activation=tanh, estimator__alpha=0.05, estimator__hidden_layer_sizes=(50, 100, 50), estimator__learning_rate=invscaling, estimator__solver=lbfgs; total time=   1.7s
[CV] END estimator__activation=tanh, estimator__alpha=0.05, estimator__hidden_layer_sizes=(50, 100, 50), estimator__learning_rate=invscaling, estimator__solver=lbfgs; total time=   1.7s
[CV] END 

## Store tuned estimators

In [9]:
# Create list where we are to store the tuned models
estimators_list = []

# Add tuned models inside list
estimators_list.append(LR_best_estimator)
estimators_list.append(RF_best_estimator)
estimators_list.append(SVM_best_estimator)
estimators_list.append(MLP_best_estimator)

# 5. Track training process

## 1. Codecarbon

In [10]:
# Track estimators_list training with codecarbon
tracking.track_benchmarking_codecabon(estimators_list, X_train, y_train)

[codecarbon INFO @ 19:33:59] [setup] RAM Tracking...
[codecarbon INFO @ 19:33:59] [setup] GPU Tracking...
[codecarbon INFO @ 19:33:59] No GPU found.
[codecarbon INFO @ 19:33:59] [setup] CPU Tracking...
[codecarbon WARNING @ 19:33:59] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 19:34:25] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:34:25] >>> Tracker's metadata:
[codecarbon INFO @ 19:34:25]   Platform system: Linux-5.19.0-38-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 19:34:25]   Python version: 3.10.8
[codecarbon INFO @ 19:34:25]   Available RAM : 4.946 GB
[codecarbon INFO @ 19:34:25]   CPU count: 4
[codecarbon INFO @ 19:34:25]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:34:25]   GPU count: None
[codecarbon INFO @ 19:34:25]   GPU model: None
[codecarbon INFO @ 19:34:29] Energy consumed for RAM : 0.000000 kWh. RAM Power : 1.8548641204833987 W
[codecarbon I

Emissions: 1.1620543876297233e-07 kg


[codecarbon INFO @ 19:34:30] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:34:30] >>> Tracker's metadata:
[codecarbon INFO @ 19:34:30]   Platform system: Linux-5.19.0-38-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 19:34:30]   Python version: 3.10.8
[codecarbon INFO @ 19:34:30]   Available RAM : 4.946 GB
[codecarbon INFO @ 19:34:30]   CPU count: 4
[codecarbon INFO @ 19:34:30]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:34:30]   GPU count: None
[codecarbon INFO @ 19:34:30]   GPU model: None
[codecarbon INFO @ 19:34:34] Energy consumed for RAM : 0.000000 kWh. RAM Power : 1.8548641204833987 W
[codecarbon INFO @ 19:34:34] Energy consumed for all CPUs : 0.000002 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:34:34] 0.000002 kWh of electricity used since the begining.
[codecarbon INFO @ 19:34:34] [setup] RAM Tracking...
[codecarbon INFO @ 19:34:34] [setup] GPU Tracking...
[codecarbon INFO @ 19:34

Emissions: 4.47549687315667e-07 kg


[codecarbon INFO @ 19:34:36] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:34:36] >>> Tracker's metadata:
[codecarbon INFO @ 19:34:36]   Platform system: Linux-5.19.0-38-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 19:34:36]   Python version: 3.10.8
[codecarbon INFO @ 19:34:36]   Available RAM : 4.946 GB
[codecarbon INFO @ 19:34:36]   CPU count: 4
[codecarbon INFO @ 19:34:36]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:34:36]   GPU count: None
[codecarbon INFO @ 19:34:36]   GPU model: None
[codecarbon INFO @ 19:34:39] Energy consumed for RAM : 0.000000 kWh. RAM Power : 1.8548641204833987 W
[codecarbon INFO @ 19:34:39] Energy consumed for all CPUs : 0.000000 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:34:39] 0.000000 kWh of electricity used since the begining.
[codecarbon INFO @ 19:34:39] [setup] RAM Tracking...
[codecarbon INFO @ 19:34:39] [setup] GPU Tracking...
[codecarbon INFO @ 19:34

Emissions: 8.055262196095076e-08 kg


[codecarbon INFO @ 19:34:41] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:34:41] >>> Tracker's metadata:
[codecarbon INFO @ 19:34:41]   Platform system: Linux-5.19.0-38-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 19:34:41]   Python version: 3.10.8
[codecarbon INFO @ 19:34:41]   Available RAM : 4.946 GB
[codecarbon INFO @ 19:34:41]   CPU count: 4
[codecarbon INFO @ 19:34:41]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 19:34:41]   GPU count: None
[codecarbon INFO @ 19:34:41]   GPU model: None
[codecarbon INFO @ 19:34:44] Energy consumed for RAM : 0.000000 kWh. RAM Power : 1.8548641204833987 W
[codecarbon INFO @ 19:34:44] Energy consumed for all CPUs : 0.000002 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 19:34:44] 0.000002 kWh of electricity used since the begining.


Emissions: 4.53650552862852e-07 kg


## 2. Eco2AI

In [11]:
# Track estimators_list training with eco2ai
tracking.track_benchmarking_eco2ai(estimators_list, X_train, y_train)

/home/misanchz/miniconda3/envs/TFG/lib/python3.10/site-packages/eco2ai/emission_track.py:135: UserWarning: 
If you use a VPN, you may have problems with identifying your country by IP.
It is recommended to disable VPN or
manually install the ISO-Alpha-2 code of your country during initialization of the Tracker() class.
You can find the ISO-Alpha-2 code of your country here: https://www.iban.com/country-codes

  warnings.warn(
/home/misanchz/miniconda3/envs/TFG/lib/python3.10/site-packages/eco2ai/tools/tools_gpu.py:39: NoGPUWarning: 

There is no any available GPU devices or your GPU is not supported by Nvidia library!
The tracker will consider CPU usage only

  warnings.warn(message="\n\nThere is no any available GPU devices or your GPU is not supported by Nvidia library!\nThe tracker will consider CPU usage only\n",
/home/misanchz/miniconda3/envs/TFG/lib/python3.10/site-packages/eco2ai/emission_track.py:135: UserWarning: 
If you use a VPN, you may have problems with identifying your c

# 6. Create benchmarking

In [12]:
benchmarking.create_benchmarking_csv()

In [13]:
df_benchmarking = preprocess.load_csv_data('benchmarking.csv')
df_benchmarking

,Herramientas,Logistic Regression(kWh),Random Forest(kWh),Support Vector Machines(kWh),Multilayer Perceptron(kWh),Eficiencia energetica
0,codecarbon,6.116076e-07,2.355525e-06,4.239612e-07,0.000002,0
1,eco2ai,5.782587e-09,7.004785e-09,4.361105e-09,0.000005,0
